In [27]:
tau = 100
C = 1000
feature_number = 20
test_size = 0.9
y_feature = 'Class'

In [28]:
import numpy as np
import time
import pandas as pd
from sklearn.model_selection import train_test_split

In [29]:
data = pd.read_csv(f'../data/credit/{feature_number}F/data.csv')
features = data.drop(columns=y_feature,axis=1)
labels = data[y_feature]
data.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V12,V13,V14,V15,V16,V17,V18,V19,V20,Class
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,0
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,0
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,0
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,0


In [30]:

from sklearn.preprocessing import MinMaxScaler

features = MinMaxScaler(feature_range=(0, np.pi)).fit_transform(features)

train_features, test_features, train_labels, test_labels = train_test_split(
    features, labels, test_size=0.9, shuffle=False
)

In [31]:
from qiskit import BasicAer
from qiskit.circuit.library import ZFeatureMap
from qiskit_algorithms.utils import algorithm_globals

from qiskit_machine_learning.kernels import FidelityQuantumKernel

algorithm_globals.random_seed = 12345

feature_map = ZFeatureMap(feature_dimension=feature_number, reps=1)

qkernel = FidelityQuantumKernel(feature_map=feature_map)

In [32]:
from qiskit_machine_learning.algorithms import PegasosQSVC

pegasos_qsvc = PegasosQSVC(quantum_kernel=qkernel, C=C, num_steps=tau)

In [33]:
start_time = time.time()
pegasos_qsvc.fit(train_features, train_labels)
elapsed_time = time.time() - start_time
print(f'Time spent to fit: {elapsed_time}' )

KeyboardInterrupt: 

In [ ]:
start_time = time.time()
predict_label = pegasos_qsvc.predict(test_features)
elapsed_time = time.time() - start_time
elapsed_time
print(f'Time spent to predict: {elapsed_time}' )

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(test_labels,predict_label))

In [ ]:
print(classification_report(test_labels,predict_label))

In [ ]:
start_time = time.time()
pegasos_score = pegasos_qsvc.score(test_features, test_labels)
elapsed_time = time.time() - start_time
print(f'score : {pegasos_score} \ntime to score: {elapsed_time}' )